# DATA

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
test = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv')

# Data Exploratory

In [ ]:
# function Exploratory Data Analysis
def eda(dfA, all=False, desc='Exploratory Data Analysis'):
    print(desc)
    print(f'\nShape:\n{dfA.shape}')
    print(f'\nDTypes - Numerics')
    print(dfA.select_dtypes(include=np.number).columns.tolist())
    print(f'\nDTypes - Categoricals')
    print(dfA.select_dtypes(include='object').columns.tolist())
    print(f'\nIs Null: {dfA.isnull().sum().sum()}')
    print(f'{dfA.isnull().mean().sort_values(ascending=False)}')
    dup = dfA.duplicated()
    print(f'\nDuplicated: \n{dfA[dup].shape}\n')
    try:
        print(dfA[dfA.duplicated(keep=False)].sample(4))
    except:
        pass
    if all:  # here you put yours prefered analysis that detail more your dataset
        
        print(f'\nDTypes - Numerics')
        print(dfA.describe(include=[np.number]))
        print(f'\nDTypes - Categoricals')
        print(dfA.describe(include=['object']))


In [ ]:
eda(train)

In [ ]:
# function Fill NaN values
def cleanNaN(dfA):
  for col in dfA:
    if type(dfA[col]) == 'object':
        dfA[col] = dfA[col].fillna('unknow')
    else:
        dfA[col] = dfA[col].fillna(0)
  return dfA

In [ ]:
treino = cleanNaN(train)

In [ ]:
eda(treino)

In [ ]:
# changing Categoricals to number
def catToNumeric(dfA):
    for x in dfA.select_dtypes(include='object').columns.tolist():
        ncol = 'cc_'+x 
        dfA[x] = pd.Categorical(dfA[x])
        dfA[ncol] = dfA[x].cat.codes
    return dfA

In [ ]:
treino2 = catToNumeric(treino)
ncols = treino2.select_dtypes(include=np.number).columns.tolist()
ncols

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff

**Correlation between variables**

In [ ]:
def correlation(dfA, varT, minValue=0.5, showGraphic=True, title='Correlation between variables'):
    corr = dfA.corr()
    print(f'\nAnalysing features:\n'
          f'Target: {varT}\n'
          f'minValue de ref.: {minValue}\n'
          f'\nMain Features:')
    corrs = corr[varT]
    features = []
    for i in range(0, len(corrs)):
        if corrs[i] > minValue and corrs.index[i] != varT:
            print(corrs.index[i], f'{corrs[i]:.2f}')
            features.append(corrs.index[i])
    if showGraphic:     
        z_text = np.around(corr.values, decimals=2) 
        fig2 = ff.create_annotated_heatmap(corr.values, 
                                          annotation_text=z_text,
                                           x=corr.index.values.tolist(),
                                y=corr.columns.values.tolist(),
                                colorscale='Viridis',
                                hoverongaps = False)
        
        for i in range(len(fig2.layout.annotations)):
            fig2.layout.annotations[i].font.size = 8
            
        fig2.show()

        
        
    
    return features

In [ ]:
varTarget = 'target'

In [ ]:
varsFeatures = correlation(treino2, varT=varTarget, minValue=-0.999)

# Regressors

In [ ]:
# ML Algorithms sklearn
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, PoissonRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyRegressor


In [ ]:
regressors = [
        DecisionTreeRegressor(),
        RandomForestRegressor(),
        SVR(),
        LinearRegression(),
        GradientBoostingRegressor(),
        PoissonRegressor(),
        DummyRegressor(),
        LogisticRegression(),
        GaussianNB()
    ]

In [ ]:
teste = cleanNaN(test)

In [ ]:
teste2 = catToNumeric(teste)

In [ ]:
Xtreino = treino2[varsFeatures]
ytreino = treino2[varTarget]

In [ ]:
Xteste = teste2[varsFeatures]

In [ ]:
reg = []
sco = []
for regressor in regressors:
    modelo = regressor
    modelo.fit(Xtreino, np.array(ytreino))
    sco.append(round(modelo.score(Xtreino, ytreino),2))
    previsao = modelo.predict(Xteste)
    reg.append(regressor)

In [ ]:
meuMae = pd.DataFrame(columns=['Regressor', 'score'])
meuMae['Regressor'] = reg
meuMae['score'] = sco

In [ ]:
meuMae = meuMae.sort_values(by='score', ascending=False)
meuMae

In [ ]:
f'Best Regressor: {meuMae["Regressor"].values[0]}'

In [ ]:
modelo = DecisionTreeRegressor()
modelo.fit(Xtreino, np.array(ytreino))
previsao = modelo.predict(Xteste)

previsao

In [ ]:
submission = pd.DataFrame()
submission['enrollee_id'] = Xteste.enrollee_id
submission['target'] = previsao

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

# Basic... but done!